In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY
from gptchem.gpt_classifier import GPTClassifier
from gptchem.tuner import Tuner

In [5]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
        "matbench_glass",
        "matbench_expt_is_metal",
    ],
)

Fetching matbench_expt_is_metal.json.gz from https://ml.materialsproject.org/projects/matbench_expt_is_metal.json.gz to /Users/kevinmaikjablonka/miniconda3/envs/gptchem/lib/python3.9/site-packages/matminer/datasets/matbench_expt_is_metal.json.gz


Fetching https://ml.materialsproject.org/projects/matbench_expt_is_metal.json.gz in MB: 0.034816MB [00:00, 46.67MB/s]                 

2023-01-17 12:22:10 INFO     Initialized benchmark 'matbench_v0.1' with 2 tasks: 
['matbench_glass', 'matbench_expt_is_metal']


In [3]:
classifier = GPTClassifier('glass', Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False))

In [7]:
t = list(mb.tasks)[0]
f, l = t.get_train_and_val_data(0)

In [9]:
classifier.fit(f, l.astype(int))

Upload progress: 100%|██████████| 501k/501k [00:00<00:00, 666Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/03_classification/matbench/out/20230117_122204/train.jsonl: file-LRyTVCX2xpAvS5p1e2h92pmb


In [16]:
for task in mb.tasks:
    task.load()

    for fold_ind, fold in enumerate(task.folds):
        print(type(fold))

        train_inputs, train_outputs = task.get_train_and_val_data(fold)

        # train and validate your model
        my_model.train_and_validate(train_inputs, train_outputs)

        # Get testing data
        test_inputs = task.get_test_data(fold, include_target=False)

        # Predict on the testing data
        # Your output should be a pandas series, numpy array, or python iterable
        # where the array elements are floats or bools
        predictions = my_model.predict(test_inputs)

        # Record your data!
        task.record(fold, predictions)
        
    if task.metadata.task_type == CLF_KEY:
        print(f"{task.dataset_name}: Accuracy score {task.scores['accuracy']['mean']}")
        print(f"{task.dataset_name}: ROC score {task.scores['rocauc']['mean']}")

    task.df = None

2023-01-17 12:11:34 INFO     Dataset matbench_glass already loaded; not reloading dataset.
<class 'int'>


TypeError: int() argument must be a string, a bytes-like object or a number, not '_NoValueType'

In [11]:
test_inputs

NameError: name 'test_inputs' is not defined